In [152]:
import load
import numpy as np
import matplotlib
matplotlib.use("Qt5Agg")
import matplotlib.pyplot as plt
import tree
import make_gal

In [153]:
class Simplemock():
    def __init__(self, repo="/home/hoseung/Work/pyclusterevol/repo/sed/",
                 filter_system="SDSS",
                 sed_model="bc03",
                 load=True):
        self.filter_system = filter_system
        self.repo = repo
        self.sed_model = sed_model
        if load is True:
            self.load_filters()
            self.load_SED_wavelength()
            self.load_SED_all()

    def load_filters(self):        
        if self.filter_system == "SDSS":
            filter_lambda, filter_u, filter_g, filter_r, filter_i, filter_z = \
                np.genfromtxt(self.repo + "filter_sdss.dat",
                                skip_header=1, unpack=True)
        
            self.filters = {"lambda":filter_lambda,
                            "u":filter_u,
                            "g":filter_g,
                            "r":filter_r,
                            "i":filter_i,
                            "z":filter_z}    

    def load_SED_wavelength(self):
        if self.sed_model == "bc03":
            self.sed_wavelength = np.genfromtxt(self.repo + "lambda.dat")
            
    def load_SED_all(self):
        """
        Full SEDs are just a few tens of MB.
        """
        if self.sed_model == "bc03":
            self.metal_points = np.array([0.0004, 0.001, 0.004, 0.01, 0.02, 0.04])
            # age points in tables.
            self.age_points = np.genfromtxt(self.repo+"ages_yybc.dat") # in Gry unit
            self.SEDs = np.zeros((6, 221, 1221))
            
            for i, metal in enumerate(self.metal_points):
                self.SEDs[i,:,:] = np.genfromtxt(self.repo +
                                            "bc03_yy_{:.4f}".format(metal)).reshape(221, 1221)
        else:
            print("Sorry, Only bc03 is implemented.")
            
    def get_flux(self, star,
                 extinction = False,
                 metal_lower_cut = True,
                 filter_name='r'):
        ### star data ########################################################
        
        starmetal = star["metal"] # Is the original array modified?
        if metal_lower_cut:
            # No star with metallicity lower than the lowest table.
            starmetal[starmetal < min(self.metal_points)] = min(self.metal_points) * 1.0001 

        locate_metal = np.digitize(starmetal, self.metal_points)-1 # GOOD
        relevant_metals = self.metal_points[:max(locate_metal)+2]
        nmetals = len(relevant_metals)

        # Star Age
        t_univ = 13.7
        starage = t_univ - gg1.star["time"]

        locate_age = np.digitize(starage, self.age_points)-1 # GOOD
        relevant_ages = self.age_points[:max(locate_age)+2]
        nages = len(relevant_ages)
        
        ### Filter optimization. #################################################

        # Pick one
        this_filter = self.filters[filter_name]

        # band range
        i_filter_pos = this_filter > 0

        this_filter = this_filter[i_filter_pos]
        filter_lambda_this_band = self.filters["lambda"][i_filter_pos]

        lambda_min_this_band = min(filter_lambda_this_band)
        lambda_max_this_band = max(filter_lambda_this_band)

        i_lambda_min = np.argmax(self.sed_wavelength > lambda_min_this_band) -1
        #print(i_lambda_min, wavelength[i_lambda_min], lambda_min_this_band)
        i_lambda_max = np.argmax(self.sed_wavelength > lambda_max_this_band)
        #print(i_lambda_max, wavelength[i_lambda_max], lambda_max_this_band)

        # Only a small part of SED is needed.
        wavelength = self.sed_wavelength[i_lambda_min:i_lambda_max+1]
        n_wavelength = i_lambda_max - i_lambda_min + 1

        # Wavelengths at which filter function are defined are different from the SED wavelength points.
        # Interpolate filter function on SED points.
        filter_in_sed_wavelengths = np.interp(wavelength, filter_lambda_this_band, this_filter)
        
        ##### Caclulate band flux #################
        # Load only necessary data
        # Load all once, keep under the class and copy part of it when needed heere. 
        seds = np.zeros((nmetals, nages, n_wavelength)) # metal age lambda
        if self.sed_model == "bc03":
            for i, metal in enumerate(relevant_metals):
                seds[i,:,:] = self.SEDs[i,:nages, i_lambda_min:i_lambda_max+1]

        # All are array-wise calculations.
        # interpolation weight
        dl_m = (starmetal - relevant_metals[locate_metal] ) / \
                                              (relevant_metals[locate_metal+1] - relevant_metals[locate_metal])
        dr_m = (relevant_metals[locate_metal+1] - starmetal) / \
                                              (relevant_metals[locate_metal+1] - relevant_metals[locate_metal])
        dl_a = (starage - relevant_ages[locate_age] )   / \
                                              (relevant_ages[locate_age+1] - relevant_ages[locate_age])
        dr_a = (relevant_ages[locate_age+1] - starage ) / \
                                              (relevant_ages[locate_age+1] - relevant_ages[locate_age])


        # 2D linear interpolation
        # weight * SED.
        Flux =  np.multiply( (dr_m * dr_a), seds[locate_metal, locate_age,:].T).T +\
                np.multiply( (dl_m * dr_a), seds[locate_metal + 1, locate_age,:].T).T +\
                np.multiply( (dr_m * dl_a), seds[locate_metal, locate_age + 1, :].T).T +\
                np.multiply( (dl_m * dl_a), seds[locate_metal + 1, locate_age + 1,:].T).T

        # Convolve filter
        Flux = np.multiply(filter_in_sed_wavelengths, Flux)

        if not extinction:
            return np.sum(Flux, axis=1)
        else:
            print("Extinction - Not yet implemented")
            return

In [2]:
cd ~/Work/data/Horizon-AGN/

/home/hoseung/Work/data/Horizon-AGN


In [173]:
cd ~/Work/data/NH/

/home/hoseung/Work/data/NH


In [ ]:
nout = 312 # 782
gcat = tree.halomodule.Halo(nout=nout, is_gal=True)

In [208]:
gcat.data.dtype

dtype((numpy.record, [('np', '<i4'), ('id', '<i4'), ('level', '<i4'), ('host', '<i4'), ('sub', '<i4'), ('nsub', '<i4'), ('nextsub', '<i4'), ('m', '<f4'), ('mvir', '<f4'), ('r', '<f4'), ('rvir', '<f4'), ('tvir', '<f4'), ('cvel', '<f4'), ('x', '<f4'), ('y', '<f4'), ('z', '<f4'), ('vx', '<f4'), ('vy', '<f4'), ('vz', '<f4'), ('ax', '<f4'), ('ay', '<f4'), ('az', '<f4'), ('sp', '<f4'), ('idx', '<i4'), ('p_rho', '<f4'), ('p_c', '<f4'), ('energy', '<f8', (3,)), ('radius', '<f8', (4,)), ('sig', '<f4'), ('sigbulge', '<f4'), ('mbulge', '<f4'), ('hosthalo', '<i4'), ('g_nbin', '<i4'), ('g_rr', '<f4', (100,)), ('g_rho', '<f4', (100,))]))

In [220]:
from matplotlib.colors import LogNorm

fig, axs = plt.subplots(1,2)
for gdata in gcat.data[5:50]:
    gg = load.rd_GM.Gal(nout, catalog=gdata.copy(), info=s.info)
    gg.debug=False
    is_good = make_gal.mk_gal(gg, mstar_min=1e8)
    print(is_good)
    gg.reorient(pops=["star"])
    axs[0].hist2d(gg.star["x"], gg.star["y"], bins=200, norm=LogNorm())
    axs[1].hist2d(gg.star["x"], gg.star["z"], bins=200, norm=LogNorm())
    plt.savefig(str(gdata["id"]).zfill(5) + ".png")
    

info <load.info.Info object at 0x7fe14d15b1d0>
File Not Found: ./halo/HAL_00312/hal_dms_0000006
No DM data loaded
File Not Found: ./GalaxyMaker/CELL_00312/gal_cells_0000006
No CELL data loaded
gal.debug False
True
info <load.info.Info object at 0x7fe14d15b1d0>
File Not Found: ./halo/HAL_00312/hal_dms_0000007
No DM data loaded
File Not Found: ./GalaxyMaker/CELL_00312/gal_cells_0000007
No CELL data loaded
gal.debug False
True
info <load.info.Info object at 0x7fe14d15b1d0>
File Not Found: ./halo/HAL_00312/hal_dms_0000008
No DM data loaded
File Not Found: ./GalaxyMaker/CELL_00312/gal_cells_0000008
No CELL data loaded
gal.debug False
True
info <load.info.Info object at 0x7fe14d15b1d0>
File Not Found: ./halo/HAL_00312/hal_dms_0000009
No DM data loaded
File Not Found: ./GalaxyMaker/CELL_00312/gal_cells_0000009
No CELL data loaded
gal.debug False
True
info <load.info.Info object at 0x7fe14d15b1d0>
File Not Found: ./halo/HAL_00312/hal_dms_0000010
No DM data loaded
File Not Found: ./GalaxyMaker/

In [221]:
s = load.sim.Sim(nout=nout)

gg1 = load.rd_GM.Gal(nout, catalog=gcat.data[21].copy(), info=s.info)
gg1.debug=False
make_gal.mk_gal(gg1)
gg1.cal_norm_vec()
#gg1.meta.nvec

[sim._hilbert_cpulist] No AMR instance,
[sim._hilbert_cpulist] Loading one...
An AMR instance is created
 Sim
An AMR instance is created
 Sim
Simulation set up.
info <load.info.Info object at 0x7fe14cd584a8>
File Not Found: ./halo/HAL_00312/hal_dms_0000022
No DM data loaded
File Not Found: ./GalaxyMaker/CELL_00312/gal_cells_0000022
No CELL data loaded
gal.debug False


array([ 0.64 , -0.168,  0.749])

In [13]:
MockSED = Simplemock()

In [222]:
gg1.star["metal"][gg1.star["metal"] >= 0.04] = 0.04 - 1e-9
## Improve Boundary cases. !!



In [179]:
plt.hist(gg1.star["metal"])
plt.show()

In [223]:
sum(gg1.star["metal"] > 0.04)

0

In [224]:
gg1.star.shape

(2696173,)

In [225]:
Flux_u = MockSED.get_flux(star=gg1.star, filter_name='u')
Flux_g = MockSED.get_flux(star=gg1.star, filter_name='g')
Flux_r = MockSED.get_flux(star=gg1.star, filter_name='r')
Flux_i = MockSED.get_flux(star=gg1.star, filter_name='i')
Flux_z = MockSED.get_flux(star=gg1.star, filter_name='z')

MemoryError: 

In [ ]:
# Can rotate galaxy. 
gg1.reorient(pops=["star"])

In [188]:
plt.hist(Flux_g)
plt.show()

In [199]:
max(Flux_i)

1.5197070352413073

In [200]:
min(Flux_i)

0.13735842219218877

In [207]:
fig, axs = plt.subplots(2,3)

x1 = gg1.star["x"]
x2 = gg1.star["y"]

npix = 200
from matplotlib.colors import LogNorm

for ax, flux in zip(axs.ravel()[:5], [Flux_u, Flux_g, Flux_r, Flux_i, Flux_z]):
    ax.hist2d(x1,x2,
           weights=np.log10(flux+1),
           bins=npix,
           cmap=plt.cm.binary_r,
           norm=LogNorm())
    ax.set_aspect("equal")


# Try scaling Flux_x arrays to make a better composite image   
comp_img = composite_rgb(x1,x2,
                         Flux_i,
                         Flux_r,
                         Flux_g,
                         npix=npix,
                         multiply_r = 1.5,
                         multiply_g = 1.2,
                         multiply_b = 1.0,
                         log_scale=True)
axs[-1][-1].imshow(comp_img)
plt.show()

In [205]:
def composite_rgb(x,y, weight_r, weight_g, weight_b,
                  npix=100,
                  multiply_r=1.3,
                  multiply_g=1.1,
                  multiply_b=1.0,
                  log_scale = True):
    from PIL import Image

    rgbArray = np.zeros((npix,npix,3))

    rgbArray[..., 0] = np.histogram2d(x, y,
               weights=weight_r, bins=npix)[0] * multiply_r
    rgbArray[..., 1] = np.histogram2d(x,y,
               weights=weight_g, bins=npix)[0] * multiply_g
    rgbArray[..., 2] = np.histogram2d(x,y,
               weights=weight_b, bins=npix)[0] * multiply_b
    
    if log_scale:
        rgbArray = np.log10(rgbArray+1)
    rgbArray = rgbArray/rgbArray.max() * 255
    #return rgbArray
    
    img = Image.fromarray(rgbArray.astype('uint8'))
    
    return img.rotate(90)
    #img.save('myimg.jpeg')